In [ ]:
#Imports 
from bs4 import BeautifulSoup
import requests 
import time
import certifi
import pandas as pd 
from urllib.parse import urljoin
import re


url = "https://www.sportkreis-fulda-huenfeld.de/vereine/?sa=449&formstep=suche"
headers = {"User-Agent": "Mozilla/5.0"}

#The website sportkreis-fulda-huenfeld.de does not hold a full SSL-certificate.
#Therefore requests is not able to verify the connection. As a consequence, I use verify=False, 
#as I am only doing Scraping in the context of this project.
resp = requests.get(url, headers=headers, verify=False)
soup = BeautifulSoup(resp.text, "html.parser")

c:\Users\bened\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.sportkreis-fulda-huenfeld.de'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [3]:
base_url = "https://www.sportkreis-fulda-huenfeld.de/"

clubs = soup.select(".vereinslist")
scraped_data = []

for i in clubs:
    name = i.select_one("h2").get_text(strip = True)
    a_tag = i.select_one("a")
    if a_tag:
        link_detailpage = urljoin(base_url, a_tag["href"])
    else:
        link_detailpage = None


    scraped_data.append({
        "scraped_club_name" : name,
        "scraped_link_detailpage" : link_detailpage 
        })

df_scraped_data = pd.DataFrame(scraped_data)

In [4]:
df_scraped_data

,scraped_club_name,scraped_link_detailpage
0,Rasdorfer Sport-Club e.V.,https://www.sportkreis-fulda-huenfeld.de/verei...
1,SG Alemannia 1921 Kleinlüder e.V.,https://www.sportkreis-fulda-huenfeld.de/verei...
2,SG Schwarz-Weiß 1919/45 Hattenhof e.V.,https://www.sportkreis-fulda-huenfeld.de/verei...
3,Sportverein 1910 Neuhof e.V.,https://www.sportkreis-fulda-huenfeld.de/verei...
4,Sportverein Hellas 1921 Michelsrombach e.V.,https://www.sportkreis-fulda-huenfeld.de/verei...
5,SV Buchonia 1912 Flieden 1912,https://www.sportkreis-fulda-huenfeld.de/verei...
6,Tanner-Tennis-Club e.V.,https://www.sportkreis-fulda-huenfeld.de/verei...
7,TC 1993 Künzell Dicker Turm e.V.,https://www.sportkreis-fulda-huenfeld.de/verei...
8,TC Schwarz-Weiss 1990 Poppenhausen e.V.,https://www.sportkreis-fulda-huenfeld.de/verei...
9,TCB Johannisau Fulda,https://www.sportkreis-fulda-huenfeld.de/verei...


In [5]:
detail_links = df_scraped_data["scraped_link_detailpage"].tolist()

In [ ]:
club_details = []

def extract_after(block, label):
    """Extracting the text following directly after a strong-label."""
    tag = block.find("strong", string=re.compile(label))
    if not tag:
        return None

    #Text is in the next sibling node
    next_node = tag.next_sibling
    if not next_node:
        return None

    # Stripping, if it is a NavigableString 
    if isinstance(next_node, str):
        return next_node.strip().strip('"')

    # Extracting text, if it is a tag
    return next_node.get_text(strip=True).strip('"')

for url in detail_links:
    html = requests.get(url, verify=False).text
    soup = BeautifulSoup(html, "html.parser")

    block = soup.select_one("#vereinsdetails")
    if not block:
        print("Kein Block gefunden für:", url)
        continue

    details = {
        "detail_url": url,
        "strasse": extract_after(block, r"Straße"),
        "plz_ort": extract_after(block, r"PLZ"),
        "telefon": extract_after(block, r"Telefon"),
        "zielgruppe": extract_after(block, r"Zielgruppe"),
        "ansprechpartner": extract_after(block, r"Ansprechpartner"),
    }

    # E-Mail
    email = block.find("a", href=re.compile("mailto"))
    details["email"] = email["href"].replace("mailto:", "") if email else None

    # Website
    website = block.find("a", href=re.compile(r"http"))
    details["website"] = website["href"] if website else None

    club_details.append(details)


c:\Users\bened\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.sportkreis-fulda-huenfeld.de'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\bened\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.sportkreis-fulda-huenfeld.de'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\bened\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.sportkreis-fulda-huenfeld.de'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\User

In [7]:
df_club_details = pd.DataFrame(club_details)

In [8]:
df_club_details

,detail_url,strasse,plz_ort,telefon,zielgruppe,ansprechpartner,email,website
0,https://www.sportkreis-fulda-huenfeld.de/verei...,None,36169 Rasdorf,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",None,None,None
1,https://www.sportkreis-fulda-huenfeld.de/verei...,An der Lüder 2,36137 Kleinlüder,0175-1613133,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",Nico Gärtner,None,http://www.sga-kleinlueder.de
2,https://www.sportkreis-fulda-huenfeld.de/verei...,None,36119 Neuhof,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",None,None,None
3,https://www.sportkreis-fulda-huenfeld.de/verei...,None,36119 Neuhof,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",None,None,None
4,https://www.sportkreis-fulda-huenfeld.de/verei...,Finkenweg 13,36088 Hünfeld,+49151 54671340,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",Daniel Kühn,None,http://www.hellas-michelsrombach.de
5,https://www.sportkreis-fulda-huenfeld.de/verei...,Hauptstr. 19,36103 Flieden,06655 / 965218; 0176 / 10052454,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",Winfried Happ,None,http://www.svflieden.de
6,https://www.sportkreis-fulda-huenfeld.de/verei...,Ensbach 1,36142 Tann,+49171 530535329,"Jugendliche, Erwachsene",Jürgen Sauermilch,None,None
7,https://www.sportkreis-fulda-huenfeld.de/verei...,Friedenstr. 45,36093 Künzell,+496619336986,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",TC 1993 Künzell Dicker Turm e.V.,None,http://www.tc-kuenzell-dicker-turm.de
8,https://www.sportkreis-fulda-huenfeld.de/verei...,None,36163 Poppenhausen,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",None,None,None
9,https://www.sportkreis-fulda-huenfeld.de/verei...,Johannisstr. 47,36041 Fulda,+49661 71314,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",Vorstand,None,https://www.tcb-johannisau-fulda.de/


In [9]:
df_club_details = df_club_details.rename(columns={"detail_url" : "scraped_link_detailpage"})

In [10]:
df_club_details_merged = df_club_details.merge(df_scraped_data, on="scraped_link_detailpage", how="left")

In [ ]:
#Rearranging columns
df_club_details_merged = df_club_details_merged[[
    "scraped_club_name",
    "strasse", 
    "plz_ort",
    "website", 
    "ansprechpartner",
    "telefon",
    "email", 
    "zielgruppe", 
    "scraped_link_detailpage"
]]

In [12]:
df_club_details_merged

,scraped_club_name,strasse,plz_ort,website,ansprechpartner,telefon,email,zielgruppe,scraped_link_detailpage
0,Rasdorfer Sport-Club e.V.,None,36169 Rasdorf,None,None,None,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
1,SG Alemannia 1921 Kleinlüder e.V.,An der Lüder 2,36137 Kleinlüder,http://www.sga-kleinlueder.de,Nico Gärtner,0175-1613133,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
2,SG Schwarz-Weiß 1919/45 Hattenhof e.V.,None,36119 Neuhof,None,None,None,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
3,Sportverein 1910 Neuhof e.V.,None,36119 Neuhof,None,None,None,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
4,Sportverein Hellas 1921 Michelsrombach e.V.,Finkenweg 13,36088 Hünfeld,http://www.hellas-michelsrombach.de,Daniel Kühn,+49151 54671340,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
5,SV Buchonia 1912 Flieden 1912,Hauptstr. 19,36103 Flieden,http://www.svflieden.de,Winfried Happ,06655 / 965218; 0176 / 10052454,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
6,Tanner-Tennis-Club e.V.,Ensbach 1,36142 Tann,None,Jürgen Sauermilch,+49171 530535329,None,"Jugendliche, Erwachsene",https://www.sportkreis-fulda-huenfeld.de/verei...
7,TC 1993 Künzell Dicker Turm e.V.,Friedenstr. 45,36093 Künzell,http://www.tc-kuenzell-dicker-turm.de,TC 1993 Künzell Dicker Turm e.V.,+496619336986,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
8,TC Schwarz-Weiss 1990 Poppenhausen e.V.,None,36163 Poppenhausen,None,None,None,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
9,TCB Johannisau Fulda,Johannisstr. 47,36041 Fulda,https://www.tcb-johannisau-fulda.de/,Vorstand,+49661 71314,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...


In [13]:
df_club_details_merged[["plz", "ort"]] = df_club_details_merged["plz_ort"].str.split(" ", n=1, expand=True)

In [14]:
df_club_details_merged = df_club_details_merged.drop(columns="plz_ort")

In [15]:
df_club_details_merged = df_club_details_merged[[
    "scraped_club_name",
    "strasse",
    "plz", 
    "ort",
    "website", 
    "ansprechpartner",
    "telefon",
    "email", 
    "zielgruppe", 
    "scraped_link_detailpage"
]]

In [16]:
df_club_details_merged

,scraped_club_name,strasse,plz,ort,website,ansprechpartner,telefon,email,zielgruppe,scraped_link_detailpage
0,Rasdorfer Sport-Club e.V.,None,36169,Rasdorf,None,None,None,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
1,SG Alemannia 1921 Kleinlüder e.V.,An der Lüder 2,36137,Kleinlüder,http://www.sga-kleinlueder.de,Nico Gärtner,0175-1613133,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
2,SG Schwarz-Weiß 1919/45 Hattenhof e.V.,None,36119,Neuhof,None,None,None,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
3,Sportverein 1910 Neuhof e.V.,None,36119,Neuhof,None,None,None,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
4,Sportverein Hellas 1921 Michelsrombach e.V.,Finkenweg 13,36088,Hünfeld,http://www.hellas-michelsrombach.de,Daniel Kühn,+49151 54671340,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
5,SV Buchonia 1912 Flieden 1912,Hauptstr. 19,36103,Flieden,http://www.svflieden.de,Winfried Happ,06655 / 965218; 0176 / 10052454,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
6,Tanner-Tennis-Club e.V.,Ensbach 1,36142,Tann,None,Jürgen Sauermilch,+49171 530535329,None,"Jugendliche, Erwachsene",https://www.sportkreis-fulda-huenfeld.de/verei...
7,TC 1993 Künzell Dicker Turm e.V.,Friedenstr. 45,36093,Künzell,http://www.tc-kuenzell-dicker-turm.de,TC 1993 Künzell Dicker Turm e.V.,+496619336986,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
8,TC Schwarz-Weiss 1990 Poppenhausen e.V.,None,36163,Poppenhausen,None,None,None,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...
9,TCB Johannisau Fulda,Johannisstr. 47,36041,Fulda,https://www.tcb-johannisau-fulda.de/,Vorstand,+49661 71314,None,"Kinder(bis 12 Jahre), Jugendliche, Erwachsene,...",https://www.sportkreis-fulda-huenfeld.de/verei...


In [17]:
df_club_details_merged_csv = df_club_details_merged.to_csv("vereinsdaten.csv", index=False)